# ICE-NMF Hyperparameter tuning ppf2

In [1]:
#imports
import numpy as np 
import csv
import re
import sys
import math
import matplotlib.pyplot as plt
from numpy import linalg as LA
import random
from sklearn.preprocessing import normalize
from numba import njit
from collections import defaultdict
import random
from implicit_data import ImplicitData
import numpy as np
from BISGD import BISGD
from ICE_NMF import ICE_NMF
from collections import defaultdict
from EvalPrequential import EvalPrequential
from datetime import datetime
import getopt
import pandas as pd

## Functions


In [2]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def sumvector(V,x):
    result = []
    for v in V:
        temp = v + x
        result.append(temp)
    return result    

def multvector(x,V):
    result = []
    for v in V:
        temp = v * x
        result.append(temp)
    return result    

def TransformVec(V):
  
    #new = normalize(V[:,np.newaxis], axis=0).ravel()
    for i in range(len(V)):
        V[i] = sigmoid(V[i])
    D = LA.norm(V)
    D=1/D
    new = D * V
  

    return new


## Get the data

In [3]:
sys.path.append('./')
configPath = 'config'
dataPath = 'ppf2.csv'

Data = []
with open(dataPath, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        Data.append([float(row[0]),float(row[1])])


In [4]:
Data_Transpose = np.transpose(Data[:50000])
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])



num_clusters = {8,16,24}
cl_num_iterations = 10
cl_learn_rate =0.1
cl_regularization = 0.1
num_factors = 90
num_iter = 8
learn_rate = 0.3
regularization = 0.4
overlap = {0.6,0.7,0.8}



In [5]:
%%capture out

tuning = {}

for num in num_clusters:
    tuning[num] = {}
    
    
    for val in overlap:

        print("")
        print("num_clusters = ", num, ", overlap = ", val)
        model = ICE_NMF(ImplicitData([],[]), num, val, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 100)
        eval = EvalPrequential(model,stream, metrics = ["Recall@20"])

        start_recommend = datetime.now()
        print('start time', start_recommend)

        results=eval.EvaluateTime(0,stream.size, 100)

        end_recommend = datetime.now()
        print('end time', end_recommend)

        tempo = end_recommend - start_recommend

        tuning[num][val] = np.mean(results['Recall@20'])
    
        print('npmean(resuls[Recall@20])', tuning[num][val])
        print('run time', tempo)
        #print('get tuple',np.mean(results['time_get_tuple']))
        print('recommend',np.mean(results['time_recommend']))
        #print('eval_point',np.mean(results['time_eval_point']))
        print('update',np.mean(results['time_update'])) 

In [6]:
print(out)


num_clusters =  8 , overlap =  0.6
start time 2023-11-23 10:51:47.792109
end time 2023-11-23 10:53:06.826357
npmean(resuls[Recall@20]) 0.27104722792607805
run time 0:01:19.034248
recommend 0.058316431495932824
update 0.00100750319480896

num_clusters =  8 , overlap =  0.7
start time 2023-11-23 10:53:06.923808
end time 2023-11-23 10:54:32.825561
npmean(resuls[Recall@20]) 0.33458646616541354
run time 0:01:25.901753
recommend 0.05689166228574021
update 0.0011074361324310302

num_clusters =  8 , overlap =  0.8
start time 2023-11-23 10:54:32.929608
end time 2023-11-23 10:56:01.523014
npmean(resuls[Recall@20]) 0.3082706766917293
run time 0:01:28.593406
recommend 0.06192427515087271
update 0.0011076207208633422

num_clusters =  16 , overlap =  0.6
start time 2023-11-23 10:56:01.626196
end time 2023-11-23 10:58:22.666564
npmean(resuls[Recall@20]) 0.3603238866396761
run time 0:02:21.040368
recommend 0.12816133190263138
update 0.0015490673446655273

num_clusters =  16 , overlap =  0.7
start tim

In [7]:
tuning

{8: {0.6: 0.27104722792607805,
  0.7: 0.33458646616541354,
  0.8: 0.3082706766917293},
 16: {0.6: 0.3603238866396761,
  0.7: 0.3172690763052209,
  0.8: 0.30991735537190085},
 24: {0.6: 0.33516483516483514,
  0.7: 0.3182711198428291,
  0.8: 0.32388663967611336}}